In [11]:
import pandas as pd
import json

In [4]:
df = pd.read_csv('/Users/m.e.zubkova/Documents/diploma_final.csv')

In [65]:
male_names = pd.read_json('/Users/m.e.zubkova/Downloads/nlp-gender/male_names.json')
female_names = pd.read_json('/Users/m.e.zubkova/Downloads/nlp-gender/female_names.json')

In [20]:
def sex_classify(name: str):
    if name in male_names.values:
        return 'male'
    if name in female_names.values:
        return 'female'
    return 'not in list of names'

In [32]:
df['freelancer_first_name'] = df.freelancer_name.apply(lambda name: name.split()[0])

In [66]:
df['freelancer_gender'] = df.freelancer_first_name.apply(sex_classify)

Часть имен мы вручную вписали в словарь для определения пола большего числа фрилансеров.

In [67]:
# определим гендер для случаев, когда пользователи поменяли местами имя и фамилию

df.loc[
    (df['freelancer_gender'] == 'not in list of names'), 'freelancer_gender'
] = df.freelancer_name.apply(
    lambda name: sex_classify(name.split()[1]) if len(name.split()) > 1 else 'not in list of names')

In [70]:
df[df['freelancer_gender'] == 'not in list of names'].shape

# без гендера осталось 604 отзыва, их мы не будем учитывать в последующем анализе

(604, 8)

In [71]:
gendered = df[df['freelancer_gender'] != 'not in list of names']
gendered.freelancer_gender.value_counts()

female    6444
male      3781
Name: freelancer_gender, dtype: int64

In [74]:
gendered[gendered.texts == 'Нет отзыва'].freelancer_gender.value_counts()

female    911
male      334
Name: freelancer_gender, dtype: int64

In [75]:
def no_text(texts: pd.Series):
    """
    Функция подсчитывает долю отзывов без текста от всех отзывов (необходимо наличие оценки)
    """
    return len(texts[texts == 'Нет отзыва']) / len(texts)

In [76]:
gendered.groupby('freelancer_gender').agg({'texts': no_text})

# доля отзывов без текста у мужчин ниже, воспользуемся критерием хи-квадрат для проверки

,texts
freelancer_gender,
female,0.141372
male,0.088336


In [89]:
gendered['texts_existence'] = gendered.texts.apply(lambda text: text != 'Нет отзыва')
ct1 = pd.crosstab(gendered['freelancer_gender'], gendered['texts_existence'])
ct1

<ipython-input-89-50ee00758079>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gendered['texts_existence'] = gendered.texts.apply(lambda text: text != 'Нет отзыва')


texts_existence,False,True
freelancer_gender,,
female,911,5533
male,334,3447


In [90]:
from scipy.stats import chi2_contingency

In [91]:
chi2_contingency(ct1)

# тест хи-квадрат показывает, что с вероятностью 99% наличие текста отзыва в отзывах на мужчин-фрилансеров и на женщин-фрилансерок значимо различается

(62.182501680642176,
 3.130573149307869e-15,
 1,
 array([[ 784.62396088, 5659.37603912],
        [ 460.37603912, 3320.62396088]]))

In [94]:
# почистим от текстов без отзывов

with_reviews = gendered[gendered.texts != 'Нет отзыва']

In [97]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [98]:
mystem = Mystem()
russian_stopwords = stopwords.words("russian")

Installing mystem to /Users/m.e.zubkova/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


In [103]:
# базово предобработаем тексты

def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords
              and token != " "
              and token.strip() not in punctuation]

    return tokens

In [106]:
with_reviews['texts_preprocessed'] = with_reviews.texts.apply(preprocess_text)

CPU times: user 1.7 s, sys: 196 ms, total: 1.9 s
Wall time: 10.9 s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [125]:
set_of_names = set(map(lambda x: x.lower(), female_names.T.values.tolist()[0])) | set(map(lambda x: x.lower(), male_names.T.values.tolist()[0]))

In [127]:
# проверим наличие имен в текстах отзывов
with_reviews['name_in_review'] = with_reviews.texts_preprocessed.apply(lambda text: len(set(text) & set_of_names) > 0)

<ipython-input-127-640374b2dd7e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_reviews['name_in_review'] = with_reviews.texts_preprocessed.apply(lambda text: len(set(text) & set_of_names) > 0)


In [129]:
ct2 = pd.crosstab(with_reviews['freelancer_gender'], with_reviews['name_in_review'])
ct2

name_in_review,False,True
freelancer_gender,,
female,3037,2496
male,2121,1326


In [130]:
chi2_contingency(ct2)

# тест хи-квадрат показывает, что с вероятностью 99% наличие имени в отзывах на мужчин-фрилансеров и на женщин-фрилансерок значимо различается

(38.06627389047619,
 6.838196863551224e-10,
 1,
 array([[3178.08619154, 2354.91380846],
        [1979.91380846, 1467.08619154]]))

In [133]:
ct3 = pd.crosstab(with_reviews['marks'], with_reviews['name_in_review'])
ct3

name_in_review,False,True
marks,,
1,151,47
2,66,26
3,33,11
4,88,20
5,1690,1047
Пять с плюсом,3130,2671


In [134]:
chi2_contingency(ct3)

# имена в отзывах чаще встречаются в положительных отзывах

(117.0366358159197,
 1.3310762579059365e-23,
 5,
 array([[ 113.72873051,   84.27126949],
        [  52.84365256,   39.15634744],
        [  25.27305122,   18.72694878],
        [  62.03385301,   45.96614699],
        [1572.0986637 , 1164.9013363 ],
        [3332.022049  , 2468.977951  ]]))

In [ ]:
!pwd